In [ ]:
import pandas as pd
#import csv data
df=pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
df

In [ ]:
#remove milliseconds and change to datetime format, because Pandas Technical Analysis requires it
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
#Set the time as the index of the dataframe, also for pandas_ta
df.set_index("Gmt time", inplace=True)
#remove days where there was no change in price, if any
df=df[df.High!=df.Low]
#check the length of the dataframe after that, removes 11 days in this case
len(df)

In [ ]:
import pandas_ta as ta
#calculate VWAP, RSI and Bollinger Bands with pandas_ta, variables chosen by preference
df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df['RSI']=ta.rsi(df.Close, length=16)
my_bbands = ta.bbands(df.Close, length=14, std=2.0)
df=df.join(my_bbands)

In [ ]:
#create a list with as many zeroes as there are rows/days in the dataframe
VWAPsignal = [0]*len(df)
#we're going to look for 15 candles above VWAP as a first-order condition for trading
backcandles = 15

for row in range(backcandles, len(df)):
    #uptrend and downtrend = 1 unless conditions are met
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i], df.Close[i])>=df.VWAP[i]:
            dnt=0
        if min(df.Open[i], df.Close[i])<=df.VWAP[i]:
            upt=0
    #if both are 1 as defined in the beginning, no trading happens
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    #uptrend => long
    elif upt==1:
        VWAPsignal[row]=2
    #downtrend => short
    elif dnt==1:
        VWAPsignal[row]=1

#add to dataframe
df['VWAPSignal'] = VWAPsignal

In [ ]:
#checking if candles close above/below the bband and RSI < 45 or RSI > 55. Again, 2 is long, 1 is short
def TotalSignal(l):
    if (df.VWAPSignal[l]==2
        and df.Close[l]<=df['BBL_14_2.0'][l]
        and df.RSI[l]<45):
            return 2
    if (df.VWAPSignal[l]==1
        and df.Close[l]>=df['BBU_14_2.0'][l]
        and df.RSI[l]>55):
            return 1
    return 0

#Same as with the VWAP signal, create empty list with the same length in rows as the dataframe
TotSignal = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
#add to dataframe
df['TotalSignal'] = TotSignal

In [ ]:
#make sure that we have equal values for all parameters, number = trades
df[df.TotalSignal!=0].count()

In [ ]:
import numpy as np
#add points below or above the candles for when there are trading signals
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

#store the positions of the points
df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
#create new dataframe for the purpose of the graph and to limit its length
st=10400
dfpl = df[st:st+350]
dfpl.reset_index(inplace=True)
#plot candles, a VWAP line and both BBands
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                           line=dict(color='blue', width=1), 
                           name="VWAP"), 
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBU")])

#add signals as dots
fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")
fig.show()

In [ ]:
#define length of the dataframe for the purpose of backtesting
dfpl = df[:75000].copy()
import pandas_ta as ta

#calculate ATR with pandas_ta, n=7 chosen arbitrarily
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)

def SIGNAL():
    return dfpl.TotalSignal

In [ ]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        #stop-loss at 1.2(coefficient)*ATR
        slatr = 1.2*self.data.ATR[-1]
        #take-profit Profit/Loss ratio (coefficient)
        TPSLRatio = 1.5

        #exit trade if RSI gets too big or too small, even if profit is not reached
        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
                self.trades[-1].close()

        #Define stop-loss and take-profit conditions for the short and long case
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

#start with $100 cash and 1/10 leverage, no commission
bt = Backtest(dfpl, MyStrat, cash=100, margin=1/10, commission=0.00)
stat = bt.run()
stat

In [ ]:
#display the cumulative returns, profit/loss per trade
bt.plot(show_legend=False)